**RQ2: Based on past trends, what are the projected self-rated poverty levels for the next decade?**

**Importing Necessary Libraries**

In [1]:

import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib as mpl
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

**Plot and Design Setting**

In [2]:
# Color settings
colors = ["#648FFF", "#785EF0", "#A11C5D", "#FE6100", "#FFB000", "#000000", "#FFFFFF"]
colors_grad = sns.color_palette('flare_r', 12)
colors_heat1 = sns.color_palette('flare_r', as_cmap=True)
colors_heat2 = sns.diverging_palette(315, 261, s=74, l=50, center='dark', as_cmap=True)
color_bg = "#1B181C"
color_text = "#FFFFFF"

# Plot settings
mpl.rcParams['figure.dpi'] = 600
mpl.rcParams["figure.figsize"] = (19.2, 10.8)
mpl.rcParams['font.family'] = 'Roboto'
mpl.rcParams['figure.titlesize'] = 32
mpl.rcParams['axes.titlesize'] = 32
mpl.rcParams['axes.titleweight'] = 'bold'
mpl.rcParams['axes.labelsize'] = 22
mpl.rcParams['xtick.labelsize'] = 22
mpl.rcParams['ytick.labelsize'] = 22
mpl.rcParams['axes.titlepad'] = 72
mpl.rcParams['axes.labelpad'] = 10
mpl.rcParams['xtick.major.pad'] = 10
mpl.rcParams['ytick.major.pad'] = 10
mpl.rcParams['xtick.major.width'] = 0
mpl.rcParams['xtick.minor.width'] = 0
mpl.rcParams['ytick.major.width'] = 0
mpl.rcParams['ytick.minor.width'] = 0
mpl.rcParams['axes.spines.top'] = False
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.grid'] = False
mpl.rcParams['legend.title_fontsize'] = 14
mpl.rcParams['legend.fontsize'] = 12
mpl.rcParams['legend.frameon'] = False
plt.rcParams['patch.linewidth'] = 0
plt.rcParams['patch.edgecolor'] = 'none'
mpl.rcParams["figure.facecolor"] = color_bg
mpl.rcParams["axes.facecolor"] = color_bg
mpl.rcParams["savefig.facecolor"] = color_bg
mpl.rcParams['text.color'] = color_text
mpl.rcParams['axes.labelcolor'] = color_text
mpl.rcParams['xtick.color'] = color_text
mpl.rcParams['ytick.color'] = color_text
mpl.rcParams['axes.edgecolor'] = color_text

**Data Preprocessing and Extraction**

In [3]:
# Load Dataset and Data Cleaning
poverty_data = pd.read_csv("../dataset/CS132_DataSheet-graph_data.csv")
poverty_data['Year'] = poverty_data['Year'].ffill() + (poverty_data['Month Only'] / 12)

In [4]:
# Extract data
x = poverty_data['Year']
y_poor = poverty_data['Poor Respondent Count']
y_not_poor = poverty_data['Not Poor Respondent Count']
y_borderline = poverty_data['Borderline Respondent Count']

**Data Standardization**

In [5]:
# Data Split and Reshape
# Poor
X_train_poor, X_test_poor, Y_train_poor, Y_test_poor = train_test_split(x, y_poor, test_size=0.5, random_state=42)
X_train_poor = np.array(X_train_poor).reshape(-1, 1)
X_test_poor = np.array(X_test_poor).reshape(-1, 1)
Y_test_poor = np.array(Y_test_poor)

# Not Poor
X_train_not_poor, X_test_not_poor, Y_train_not_poor, Y_test_not_poor = train_test_split(x, y_not_poor, test_size=0.5, random_state=42)
X_train_not_poor = np.array(X_train_not_poor).reshape(-1, 1)
X_test_not_poor = np.array(X_test_not_poor).reshape(-1, 1)
Y_test_not_poor = np.array(Y_test_not_poor)

# Borderline
X_train_borderline, X_test_borderline, Y_train_borderline, Y_test_borderline = train_test_split(x, y_borderline, test_size=0.5, random_state=42)
X_train_borderline = np.array(X_train_borderline).reshape(-1, 1)
X_test_borderline = np.array(X_test_borderline).reshape(-1, 1)
Y_test_borderline = np.array(Y_test_borderline)

**Regrssion Model Training**

In [6]:
# Regression Models
# Poor
model_poor = linear_model.LinearRegression()
model_poor.fit(X_train_poor, Y_train_poor)
Y_pred_poor = model_poor.predict(X_test_poor)

# Not Poor
model_not_poor = linear_model.LinearRegression()
model_not_poor.fit(X_train_not_poor, Y_train_not_poor)
Y_pred_not_poor = model_not_poor.predict(X_test_not_poor)

# Borderline
model_borderline = linear_model.LinearRegression()
model_borderline.fit(X_train_borderline, Y_train_borderline)
Y_pred_borderline = model_borderline.predict(X_test_borderline)

# Polynomial Regression Models
degree = 3  # Degree of the polynomial

# Poor
poly_model_poor = make_pipeline(PolynomialFeatures(degree), linear_model.LinearRegression())
poly_model_poor.fit(X_train_poor, Y_train_poor)
y_pred_poor_poly = poly_model_poor.predict(X_test_poor)

# Not Poor
poly_model_not_poor = make_pipeline(PolynomialFeatures(degree), linear_model.LinearRegression())
poly_model_not_poor.fit(X_train_not_poor, Y_train_not_poor)
y_pred_not_poor_poly = poly_model_not_poor.predict(X_test_not_poor)

# Borderline
poly_model_borderline = make_pipeline(PolynomialFeatures(degree), linear_model.LinearRegression())
poly_model_borderline.fit(X_train_borderline, Y_train_borderline)
y_pred_borderline_poly = poly_model_borderline.predict(X_test_borderline)

**Regression Metrics**

In [7]:
# Print Regression Metrics
print("Poor:")
print(f"Coefficient: {model_poor.coef_[0]:.2f}")
print(f"Intercept: {model_poor.intercept_:.2f}")
print(f"Mean Squared Error: {mean_squared_error(Y_test_poor, Y_pred_poor):.2f}")
print(f"R-squared: {r2_score(Y_test_poor, Y_pred_poor):.2f}")
print(f"Mean Squared Error (Poly): {mean_squared_error(Y_test_poor, y_pred_poor_poly):.2f}")
print(f"R-squared (Poly): {r2_score(Y_test_poor, y_pred_poor_poly):.2f}")
print("\nNot Poor:")
print(f"Coefficient: {model_not_poor.coef_[0]:.2f}")
print(f"Intercept: {model_not_poor.intercept_:.2f}")
print(f"Mean Squared Error: {mean_squared_error(Y_test_not_poor, Y_pred_not_poor):.2f}")
print(f"R-squared: {r2_score(Y_test_not_poor, Y_pred_not_poor):.2f}")
print(f"Mean Squared Error (Poly): {mean_squared_error(Y_test_not_poor, y_pred_not_poor_poly):.2f}")
print(f"R-squared (Poly): {r2_score(Y_test_not_poor, y_pred_not_poor_poly):.2f}")
print("\nBorderline:")
print(f"Coefficient: {model_borderline.coef_[0]:.2f}")
print(f"Intercept: {model_borderline.intercept_:.2f}")
print(f"Mean Squared Error: {mean_squared_error(Y_test_borderline, Y_pred_borderline):.2f}")
print(f"R-squared: {r2_score(Y_test_borderline, Y_pred_borderline):.2f}")
print(f"Mean Squared Error (Poly): {mean_squared_error(Y_test_borderline, y_pred_borderline_poly):.2f}")
print(f"R-squared (Poly): {r2_score(Y_test_borderline, y_pred_borderline_poly):.2f}")

Poor:
Coefficient: -6.12
Intercept: 12928.45
Mean Squared Error: 4640.55
R-squared: 0.50
Mean Squared Error (Poly): 3776.55
R-squared (Poly): 0.59

Not Poor:
Coefficient: 4.05
Intercept: -7892.84
Mean Squared Error: 2608.98
R-squared: 0.52
Mean Squared Error (Poly): 2623.14
R-squared (Poly): 0.52

Borderline:
Coefficient: 2.06
Intercept: -3835.61
Mean Squared Error: 6611.97
R-squared: 0.02
Mean Squared Error (Poly): 5927.41
R-squared (Poly): 0.12


**Data Plotting**

In [ ]:
# Create the plot with ALL original event labels
plt.figure(figsize=(19.2, 10.8))


# Predict values for the extended range using polynomial models
x_data_range = np.linspace(x.min(), x.max(), 100).reshape(-1, 1)
y_pred_poor_poly = poly_model_poor.predict(x_data_range)
y_pred_not_poor_poly = poly_model_not_poor.predict(x_data_range)
y_pred_borderline_poly = poly_model_borderline.predict(x_data_range)

# Extended the range
x_extended_range = np.linspace(x.min(), x.max() + 10, 100).reshape(-1, 1)
y_pred_poor_extended_poly = poly_model_poor.predict(x_extended_range)
y_pred_not_poor_extended_poly = poly_model_not_poor.predict(x_extended_range)
y_pred_borderline_extended_poly = poly_model_borderline.predict(x_extended_range)


# --- SCATTER PLOTS (with descriptive labels) ---
scatter_poor = plt.scatter(x, y_poor, color=colors[0], alpha=0.4, label='Self-Rated Poor', s=100)
scatter_not_poor = plt.scatter(x, y_not_poor, color=colors[2], alpha=0.4, label='Self-Rated Not Poor', s=100)
scatter_borderline = plt.scatter(x, y_borderline, color=colors[4], alpha=0.4, label='Self-Rated Borderline', s=100)

# --- TREND LINES --- 
# (Keep solid/dashed distinction with original colors)
plt.plot(x_data_range, y_pred_poor_poly, color=colors[0], linestyle='-', linewidth=2)
plt.plot(x_data_range, y_pred_not_poor_poly, color=colors[2], linestyle='-', linewidth=2)
plt.plot(x_data_range, y_pred_borderline_poly, color=colors[4], linestyle='-', linewidth=2)
plt.plot(x_extended_range, y_pred_poor_extended_poly, color=colors[0], linestyle='--', linewidth=2)
plt.plot(x_extended_range, y_pred_not_poor_extended_poly, color=colors[2], linestyle='--', linewidth=2)
plt.plot(x_extended_range, y_pred_borderline_extended_poly, color=colors[4], linestyle='--', linewidth=2)

# --- REINSTATED EVENT LABELS (original exact text/positions) ---
# 1986 People Power Revolution
plt.axvline(x=1986, color=colors_grad[2], linestyle='--')
plt.annotate('1986 People Power Revolution',
             xy=(1986, 900), xytext=(1986-1, 900),
             arrowprops=dict(facecolor=colors_grad[2], arrowstyle='->', lw=0.8),
             fontsize=14,
             bbox=dict(boxstyle='round,pad=0.2', edgecolor='white', facecolor=colors_grad[2]))

# 1991 Mount Pinatubo Eruption
plt.axvline(x=1991, color=colors_grad[4], linestyle='--')
plt.annotate('1991 Mount Pinatubo Eruption',
             xy=(1991, 800), xytext=(1991-1, 800),
             arrowprops=dict(facecolor=colors_grad[4], arrowstyle='->', lw=0.8),
             fontsize=14,
             bbox=dict(boxstyle='round,pad=0.2', edgecolor='white', facecolor=colors_grad[4]))

# 1997 Asian Financial Crisis
plt.axvline(x=1997, color=colors_grad[5], linestyle='--')
plt.annotate('1997 Asian Financial Crisis',
             xy=(1997, 700), xytext=(1997-1, 700),
             arrowprops=dict(facecolor=colors_grad[5], arrowstyle='->', lw=0.8),
             fontsize=14,
             bbox=dict(boxstyle='round,pad=0.2', edgecolor='white', facecolor=colors_grad[5]))

# 2008 Global Financial Crisis
plt.axvline(x=2008, color=colors_grad[6], linestyle='--')
plt.annotate('2008 Global Financial Crisis',
             xy=(2008, 600), xytext=(2008-1, 600),
             arrowprops=dict(facecolor=colors_grad[6], arrowstyle='->', lw=0.8),
             fontsize=14,
             bbox=dict(boxstyle='round,pad=0.2', edgecolor='white', facecolor=colors_grad[6]))

# 2020 COVID 19 Pandemic
plt.axvline(x=2020, color=colors_grad[7], linestyle='--')
plt.annotate('2020 COVID 19 Pandemic',
             xy=(2020, 500), xytext=(2020-1, 500),
             arrowprops=dict(facecolor=colors_grad[7], arrowstyle='->', lw=0.8),
             fontsize=14,
             bbox=dict(boxstyle='round,pad=0.2', edgecolor='white', facecolor=colors_grad[7]))

# 2023 Post Pandemic Recovery
plt.axvline(x=2023, color=colors_grad[8], linestyle='--')
plt.annotate('2023 Post Pandemic Recovery',
             xy=(2023, 400), xytext=(2023-1, 400),
             arrowprops=dict(facecolor=colors_grad[8], arrowstyle='->', lw=0.8),
             fontsize=14,
             bbox=dict(boxstyle='round,pad=0.2', edgecolor='white', facecolor=colors_grad[8]))

# 2026 Prediction
plt.axvline(x=2025.5, color=colors_grad[10], linestyle='--')
plt.annotate('Prediction for self-rated poverty',
             xy=(2026, 900), xytext=(2026-1, 900),
             arrowprops=dict(facecolor=colors_grad[10], arrowstyle='->', lw=0.8),
             fontsize=14,
             bbox=dict(boxstyle='round,pad=0.2', edgecolor='white', facecolor=colors_grad[10]),
             color='black')

# --- LABELS (new) ---
plt.ylabel('Number of Respondents by Self-Rated Poverty Status', fontsize=18) 
plt.xlabel('Year', fontsize=20)
plt.title('Philippines Self-Rated Poverty Trends and Prediction (1986-2030)', 
          fontdict={'family': 'Roboto', 'weight': 'bold', 'size': 32}, pad=50)

# --- LEGENDS (with original colors) ---

# Firstlegend: Trends
trend_legend = plt.legend(handles=[
    plt.Line2D([0], [0], color=colors[0], linestyle='-', label='Poor Trend'),
    plt.Line2D([0], [0], color=colors[2], linestyle='-', label='Not Poor Trend'),
    plt.Line2D([0], [0], color=colors[4], linestyle='-', label='Borderline Trend'),
    plt.Line2D([0], [0], color='gray', linestyle='--', label='Projected Trend'),
],
    title='Trend Lines:',
    loc='upper left',
    bbox_to_anchor=(0.90, 0.75),
    fontsize=16
)

plt.gca().add_artist(trend_legend)

# Second legend: Data points
plt.legend(handles=[scatter_poor, scatter_not_poor, scatter_borderline],
                        title='Self-Rated Poverty:',
                        loc='best',
                        bbox_to_anchor=(0.90, 0.55),
                        fontsize=16)

plt.figtext(0.85, -0.05, 'Source: Social Weather Stations (SWS) Self Rated Poverty Data (2025)\nBy: Marc Dela Paz', ha='center', va='center', fontsize=12)



plt.subplots_adjust(right=1.05, top=0.95, left=0.05, bottom=0.05)
plt.grid(True, linestyle='--', alpha=0.3)
plt.savefig('../plot_results/nutshell_plot.pdf', bbox_inches='tight')
plt.show()

findfont: Font family 'Roboto' not found.
findfont: Font family 'Roboto' not found.
findfont: Font family 'Roboto' not found.
findfont: Font family 'Roboto' not found.
findfont: Font family 'Roboto' not found.
findfont: Font family 'Roboto' not found.
findfont: Font family 'Roboto' not found.
findfont: Font family 'Roboto' not found.
findfont: Font family 'Roboto' not found.
findfont: Font family 'Roboto' not found.
findfont: Font family 'Roboto' not found.
findfont: Font family 'Roboto' not found.
findfont: Font family 'Roboto' not found.
findfont: Font family 'Roboto' not found.
findfont: Font family 'Roboto' not found.
findfont: Font family 'Roboto' not found.
findfont: Font family 'Roboto' not found.
findfont: Font family 'Roboto' not found.
findfont: Font family 'Roboto' not found.
findfont: Font family 'Roboto' not found.
findfont: Font family 'Roboto' not found.
findfont: Font family 'Roboto' not found.
findfont: Font family 'Roboto' not found.
findfont: Font family 'Roboto' not